In [1]:
import os
import time
import typing
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Concatenate,
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    PointCloud,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
    ConstantVectorSource
)

from manipulation import running_as_notebook
# from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation, AddPointClouds
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph

from utils import *
from scene_utils import *

meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7000


In [2]:
drivers = f"""
model_drivers:
    iiwa: !IiwaDriver
        control_mode: position_only
        hand_model_name: wsg
"""

directives = f"""
directives:
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
        default_joint_positions:
            iiwa_joint_1: [-1.57]
            iiwa_joint_2: [0.1]
            iiwa_joint_3: [0]
            iiwa_joint_4: [-1.2]
            iiwa_joint_5: [0]
            iiwa_joint_6: [ 1.6]
            iiwa_joint_7: [0]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0
        X_PC:
            translation: [0, -1, 0]
            rotation: !Rpy {{ deg: [0, 0, 180] }}
    - add_model:
        name: probe
        file: package://stackbot/probe.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: probe::probe_link
        X_PC:
            translation: [0, 0, 0.14]
            rotation: !Rpy {{deg: [0, 0, 0]}}
    - add_model:
        name: floor
        file: package://stackbot/floor.sdf
    - add_weld:
        parent: world
        child: floor::floor_link
        X_PC:
            translation: [0, 0, -0.05]
    - add_model:
        name: platform
        file: package://stackbot/platform.sdf
    - add_weld:
        parent: world
        child: platform::platform_link
"""

block_directives = create_block_directives_from_file("../assets/contexts/stack_rrt_114.json")
scenario_yaml = drivers + directives + block_directives

In [3]:
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(
    scenario=scenario,
    meshcat=meshcat,
    package_xmls=["../assets/models/package.xml"],
)
plant = station.GetSubsystemByName("plant")

builder = DiagramBuilder()
builder.AddSystem(station)

initial_iiwa_pos = builder.AddSystem(ConstantVectorSource(np.zeros(station.GetInputPort("iiwa.position").size())))
builder.Connect(initial_iiwa_pos.get_output_port(), station.GetInputPort("iiwa.position"))

diagram = builder.Build()

In [4]:
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
simulator = Simulator(diagram, diagram_context)

# run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5)
meshcat.StopRecording()
meshcat.PublishRecording()

In [5]:
def avg_translation_error(filename):
    with open(filename, "r") as f:
        block_poses = json.load(f)
    total_offset = 0
    for block in block_poses.values():
        translation = block["translation"]
        total_offset += (translation[0] ** 2 + translation[1] ** 2) ** 0.5
    return total_offset / len(block_poses)

In [6]:
avg_translation_error("../assets/contexts/stack_rrt_114.json")

0.47557793646109836

In [7]:
def avg_z_rotation_error(filename, target_rot):
    with open(filename, "r") as f:
        block_poses = json.load(f)
    total_error = 0
    for block in block_poses.values():
        theta = block["rotation"][2]
        # Wrap angle into [-pi, pi]
        d = ((theta - target_rot + np.pi) % (2*np.pi)) - np.pi
        # Account for 180-degree symmetry
        d = abs(d)
        d = min(d, abs(d - np.pi))
        print(theta, d)
        total_error += d
    return total_error / len(block_poses)

In [8]:
avg_z_rotation_error("../assets/contexts/stack_diffIK_114.json", 0)

-3.1382738922850564 0.0033187613047367392
-0.018339639596359963 0.01833963959635998
3.13400623919296 0.007586414396833163
0.04021795957558001 0.04021795957558005


0.017365693718377484

In [9]:
avg_z_rotation_error("../assets/contexts/stack_rrt_114.json", 0)

-1.7104226669544433 1.4311699866353498
1.3439035240356338 1.3439035240356336
-2.146754979953025 0.9948376736367681
2.2689280275926285 0.8726646259971647


1.160643952576229